In [14]:
import numpy as np
import pandas as pd
import torch

from utlis import train_test_split_data
from datasets import Dataset

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

SEED = 42
set_seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == "cuda":
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "mcc": matthews_corrcoef(labels, preds),
    }


### Loading and preparing Data

In [17]:
df = pd.read_pickle("../data/combined_df.pkl")
df["content_text_joined"] = df["content_words"].apply(lambda x: " ".join(x))
TEXT_COL = "content_text_joined"    
LABEL_COL = "is_spoiler"      

# Split
X_train, X_test, y_train, y_test = train_test_split_data(
    df,
    TEXT_COL,
    LABEL_COL
)

train_ds = Dataset.from_dict({
    "text": X_train,
    "labels": y_train.astype(int)
})

test_ds = Dataset.from_dict({
    "text": X_test,
    "labels": y_test.astype(int)
})

# validation split from train
split_ds = train_ds.train_test_split(test_size=0.2, seed=SEED)

### Training

In [18]:
MODEL_CHECKPOINT = "bert-base-uncased"
NUM_LABELS = 2

In [19]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [20]:
MAX_LENGTH = 256  # conscious trade-off

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH
    )

tokenized_ds = {
    "train": split_ds["train"].map(tokenize, batched=True, remove_columns=["text"]),
    "val": split_ds["test"].map(tokenize, batched=True, remove_columns=["text"]),
}

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/56000 [00:00<?, ? examples/s]

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

In [21]:
# freeze all layers 
for param in model.bert.parameters():
    param.requires_grad = False

# unfreeze last 2 encoder layers
for layer in model.bert.encoder.layer[-2:]:
    for param in layer.parameters():
        param.requires_grad = True

# unfreeze classifier
for param in model.classifier.parameters():
    param.requires_grad = True


In [22]:
training_args = TrainingArguments(
    output_dir="./results/BERT",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,          # IMPORTANT: not 5
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="mcc",
    greater_is_better=True,
    logging_dir="./logs/BERT",
    report_to="none",
    seed=SEED,
)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\User\AppData\Local\Temp\ipykernel_25320\2330921089.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
trainer.train()

  0%|          | 0/10500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [19]:
trainer.save_model("../Models/Bert/final_model")
tokenizer.save_pretrained("../Models/Bert/tokenizer")

('../Models/Bert/tokenizer\\tokenizer_config.json',
 '../Models/Bert/tokenizer\\special_tokens_map.json',
 '../Models/Bert/tokenizer\\vocab.txt',
 '../Models/Bert/tokenizer\\added_tokens.json',
 '../Models/Bert/tokenizer\\tokenizer.json')